In [ ]:
  !ls -lrt
#download data
print("Downloading Dataset:")
!wget --quiet http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip

total 41828
-rw-r--r-- 1 root root  8324970 Aug 23 14:57 deu-eng.zip
-rw-r--r-- 1 root root 34494242 Aug 23 23:57 deu.txt
-rw-r--r-- 1 root root     1441 Aug 23 23:57 _about.txt
drwxr-xr-x 1 root root     4096 Oct  5 16:31 sample_data
Archive:  deu-eng.zip
replace deu.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: deu.txt                 
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _about.txt              


In [ ]:
ls -lrt

total 49960
-rw-r--r-- 1 root root  8324970 Aug 23 14:57 deu-eng.zip.1
-rw-r--r-- 1 root root  8324970 Aug 23 14:57 deu-eng.zip
-rw-r--r-- 1 root root 34494242 Aug 23 23:57 deu.txt
-rw-r--r-- 1 root root     1441 Aug 23 23:57 _about.txt
drwxr-xr-x 1 root root     4096 Oct  5 16:31 sample_data/


In [ ]:
import csv
import string
import re
from typing import List, Tuple
from pickle import dump
from unicodedata import normalize
import numpy as np
import itertools
from pickle import load
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from pickle import load
import random
import tensorflow as tf
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
import numpy as np

In [ ]:
# Start of sentence
SOS = "<start>"
# End of sentence
EOS = "<end>"
# Relavent Punctuation
PUNCTUATION = set("?,!,.")

In [ ]:
def load_dataset(filename: str) -> str:
  """
  load dataset into memory
  """
  with open(filename, mode="rt", encoding="utf-8") as fp:
    return fp.read()

In [ ]:
def to_pairs(dataset: str, limit: int=None, shuffle=False) -> List[Tuple[str, str]]:
  """
  Split dataset into pair of sentences
  :param dataset: dataset containing examples of translations between two languages
  the examples are delimited by `\n` and the contents of the lines are
  delimited by `\t`
  :param limit: number that limit dataset size (optional)
  :param shuffle: default is True
  :return: list of pairs
  """
  assert isinstance(limit, (int, type(None))), TypeError(
      "The limit value must be an integer"
  )
  lines = dataset.strip().split('\n')
  # Random Dataset
  if shuffle is True:
    random.shuffle(lines)
  number_examples = limit or len(lines)
  pairs = []
  for line in lines[: abs(number_examples)]:
    # take only source and target
    src, trg, _ = line.split("\t")
    pairs.append((src, trg))

  # dataset size check
  assert len(pairs) == number_examples
  return pairs

In [ ]:
def separe_punctuation(token: str) -> str:
  """
  Separe punctuation if they exist
  """
  if not set(token).intersection(PUNCTUATION):
    return token
  for p in PUNCTUATION:
    token = f" {p} ".join(token.split(p))
  return " ".join(token.split(p))    

In [ ]:
def preprocess(sentence: str, add_start_end: bool=True) -> str:
  """
  - convert lowercase
  - remove numbers
  - remove speacial characters
  - separe punctuation
  - add <start> and <end> of sentence
  :param add_start_end: add SOS and EOS
  """
  re_print = re.compile(f"[^{re.escape(string.printable)}]")
  # convert lowercase and normalizing unicode characters
  sentence = (
      normalize("NFD", sentence.lower()).encode("ascii", "ignore").decode("UTF-8")
  )
  cleaned_tokens = []
  # tokenize sentence on white space
  for token in sentence.split():
    # remove non printable characters from each token
    token = re_print.sub("", token).strip()
    # ignore token with numbers
    if re.findall("[0-9]", token):
      continue
    # add space between words and punctuation
    token = separe_punctuation(token)
    cleaned_tokens.append(token)

  # rebuild sentence with space between tokens
  sentence = " ".join(cleaned_tokens)

  # add a start and end token to the sentence
  if add_start_end is True:
    sentence = f"{SOS} {sentence} {EOS}"
  return sentence      

In [ ]:
def dataset_preprocess(dataset: List[Tuple[str, str]]) -> Tuple[List[str], List[str]]:
  """
  Returns processed dataset

  :param dataset: list of sentence pairs
  :return: list of prallel data
  """
  source_cleaned = []
  target_cleaned = []
  for source, target in dataset:
    source_cleaned.append(preprocess(source))
    target_cleaned.append(preprocess(target))
  return source_cleaned, target_cleaned  


In [ ]:
NUM_EXAMPLES = 10000 # Limit dataset size
filename = "deu.txt"
dataset = load_dataset(filename)
# get pairs limited to 1000 pairs
pairs = to_pairs(dataset, limit=NUM_EXAMPLES)
print(f"Dataset size: {len(pairs)}")
raw_data_en, raw_data_ge = dataset_preprocess(pairs)

# show last 5 pairs
for pair in zip(raw_data_en[-5:],raw_data_ge[-5:]):
    print(pair)

Dataset size: 10000
('<start> tom was crying    <end>', '<start> tom flennte    <end>')
('<start> tom was eating    <end>', '<start> tom hat gegessen    <end>')
('<start> tom was famous    <end>', '<start> tom war beruhmt    <end>')
('<start> tom was framed    <end>', '<start> tom wurde reingelegt    <end>')
('<start> tom was fuming    <end>', '<start> tom war wutend    <end>')


In [ ]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
en_tokenizer.fit_on_texts(raw_data_en)

data_en = en_tokenizer.texts_to_sequences(raw_data_en)
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en, padding='post')

ge_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
ge_tokenizer.fit_on_texts(raw_data_ge)

data_ge = ge_tokenizer.texts_to_sequences(raw_data_ge)
data_ge = tf.keras.preprocessing.sequence.pad_sequences(data_ge, padding='post')

In [ ]:
def max_len(tensor):
    #print( np.argmax([len(t) for t in tensor]))
    return max( len(t) for t in tensor)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data_en, data_ge, test_size=0.2)
BATCH_SIZE = 64
BUFFER_SIZE = len(X_train)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 256
rnn_units = 1024
dense_units = 1024
Dtype = tf.float32 #used to initialize DecoderCell Zero State

In [ ]:
Tx = max_len(data_en)
Ty = max_len(data_ge)

input_vocab_size = len(en_tokenizer.word_index)+1
output_vocab_size = len(ge_tokenizer.word_index)+ 1
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape)
print(example_Y.shape)

(64, 8)
(64, 12)


In [ ]:
# ENCODER
class EncoderNetwork(tf.keras.Model):
  def __init__(self, input_vocab_size, embedding_dims, rnn_units):
    super().__init__()
    self.encoder_embedding = tf.keras.layers.Embedding(input_dim=input_vocab_size, output_dim=embedding_dims)
    self.encoder_rnnlayer = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)


# DECODER
class DecoderNetwork(tf.keras.Model):
  def __init__(self, output_vocab_size, embedding_dims, rnn_units):
    super().__init__()
    self.decoder_embedding = tf.keras.layers.Embedding(input_dim=output_vocab_size, output_dim=embedding_dims)
    self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
    self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
    # Sampler
    self.sampler = tfa.seq2seq.sampler.TrainingSampler()
    # Create attention mechanism with memory = None
    self.attention_mechanism = self.build_attention_mechanism(dense_units, None, BATCH_SIZE*[Tx])
    self.rnn_cell = self.build_rnn_cell(BATCH_SIZE)
    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.dense_layer)

  def build_attention_mechanism(self, units, memory, memory_sequence_length):
    return tfa.seq2seq.LuongAttention(units, memory=memory, memory_sequence_length=memory_sequence_length)
    # tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)


    # wrap decoderrnn cell
  def build_rnn_cell(self, batch_size):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism, attention_layer_size=dense_units)
    return rnn_cell

  def build_decoder_initial_state(self, batch_size, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_size, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state

encoderNetwork = EncoderNetwork(input_vocab_size, embedding_dims, rnn_units)
decoderNetwork = DecoderNetwork(output_vocab_size, embedding_dims, rnn_units)
optimizer = tf.keras.optimizers.Adam()



In [ ]:
def loss_function(y_pred, y):
   
    #shape of y [batch_size, ty]
    #shape of y_pred [batch_size, Ty, output_vocab_size] 
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    mask = tf.logical_not(tf.math.equal(y,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

def train_step(input_batch, output_batch,encoder_initial_cell_state):
    #initialize loss = 0
    loss = 0
    with tf.GradientTape() as tape:
        encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
        a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                        initial_state =encoder_initial_cell_state)

        #[last step activations,last memory_state] of encoder passed as input to decoder Network
        
         
        # Prepare correct Decoder input & output sequence data
        decoder_input = output_batch[:,:-1] # ignore <end>
        #compare logits with timestepped +1 version of decoder_input
        decoder_output = output_batch[:,1:] #ignore <start>


        # Decoder Embeddings
        decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

        #Setting up decoder memory from encoder output and Zero State for AttentionWrapperState
        decoderNetwork.attention_mechanism.setup_memory(a)
        decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                           encoder_state=[a_tx, c_tx],
                                                                           Dtype=tf.float32)
        
        #BasicDecoderOutput        
        outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                               sequence_length=BATCH_SIZE*[Ty-1])

        logits = outputs.rnn_output
        #Calculate loss

        loss = loss_function(logits, decoder_output)

    #Returns the list of all layer variables / weights.
    variables = encoderNetwork.trainable_variables + decoderNetwork.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss

In [ ]:
#RNN LSTM hidden and memory state initializer
def initialize_initial_state():
  return [tf.zeros((BATCH_SIZE, rnn_units)), tf.zeros((BATCH_SIZE, rnn_units))]

In [ ]:
epochs = 15
for i in range(1, epochs+1):

    encoder_initial_cell_state = initialize_initial_state()
    total_loss = 0.0

    for ( batch , (input_batch, output_batch)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_batch, output_batch, encoder_initial_cell_state)
        total_loss += batch_loss
        if (batch+1)%5 == 0:
            print("total loss: {} epoch {} batch {} ".format(batch_loss.numpy(), i, batch+1))

total loss: 4.120637893676758 epoch 1 batch 5 
total loss: 2.7828941345214844 epoch 1 batch 10 
total loss: 2.3763608932495117 epoch 1 batch 15 
total loss: 2.2228238582611084 epoch 1 batch 20 
total loss: 2.1833066940307617 epoch 1 batch 25 
total loss: 2.0061843395233154 epoch 1 batch 30 
total loss: 1.9310638904571533 epoch 1 batch 35 
total loss: 1.987554669380188 epoch 1 batch 40 
total loss: 1.8907928466796875 epoch 1 batch 45 
total loss: 1.8459843397140503 epoch 1 batch 50 
total loss: 1.90363347530365 epoch 1 batch 55 
total loss: 1.8472965955734253 epoch 1 batch 60 
total loss: 2.021408796310425 epoch 1 batch 65 
total loss: 1.8862862586975098 epoch 1 batch 70 
total loss: 1.761630654335022 epoch 1 batch 75 
total loss: 1.7737637758255005 epoch 1 batch 80 
total loss: 1.7739166021347046 epoch 1 batch 85 
total loss: 1.609162449836731 epoch 1 batch 90 
total loss: 1.7197016477584839 epoch 1 batch 95 
total loss: 1.71544349193573 epoch 1 batch 100 
total loss: 1.713951110839843

In [ ]:
#In this section we evaluate our model on a raw_input converted to german, for this the entire sentence has to be passed
#through the length of the model, for this we use greedsampler to run through the decoder
#and the final embedding matrix trained on the data is used to generate embeddings
input_raw='how are you'

# We have a transcript file containing English-German pairs
# Preprocess X
input_raw = preprocess(input_raw, add_start_end=False)
input_lines = [f'{SOS} {input_raw}']
input_sequences = [[en_tokenizer.word_index[w] for w in line.split()] for line in input_lines]
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences,
                                                                maxlen=Tx, padding='post')
inp = tf.convert_to_tensor(input_sequences)
#print(inp.shape)
inference_batch_size = input_sequences.shape[0]
encoder_initial_cell_state = [tf.zeros((inference_batch_size, rnn_units)),
                              tf.zeros((inference_batch_size, rnn_units))]
encoder_emb_inp = encoderNetwork.encoder_embedding(inp)
a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp,
                                                initial_state =encoder_initial_cell_state)
print('a_tx :', a_tx.shape)
print('c_tx :', c_tx.shape)

start_tokens = tf.fill([inference_batch_size],ge_tokenizer.word_index[SOS])

end_token = ge_tokenizer.word_index[EOS]

greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

decoder_input = tf.expand_dims([ge_tokenizer.word_index[SOS]]* inference_batch_size,1)
decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

decoder_instance = tfa.seq2seq.BasicDecoder(cell = decoderNetwork.rnn_cell, sampler = greedy_sampler,
                                            output_layer=decoderNetwork.dense_layer)
decoderNetwork.attention_mechanism.setup_memory(a)
#pass [ last step activations , encoder memory_state ] as input to decoder for LSTM
print(f"decoder_initial_state = [a_tx, c_tx] : {np.array([a_tx, c_tx]).shape}")
decoder_initial_state = decoderNetwork.build_decoder_initial_state(inference_batch_size,
                                                                   encoder_state=[a_tx, c_tx],
                                                                   Dtype=tf.float32)
print(f"""
Compared to simple encoder-decoder without attention, the decoder_initial_state
is an AttentionWrapperState object containing s_prev tensors and context and alignment vector

decoder initial state shape: {np.array(decoder_initial_state).shape}
decoder_initial_state tensor
{decoder_initial_state}
""")

# Since we do not know the target sequence lengths in advance, we use maximum_iterations to limit the translation lengths.
# One heuristic is to decode up to two times the source sentence lengths.
maximum_iterations = tf.round(tf.reduce_max(Tx) * 2)

#initialize inference decoder
decoder_embedding_matrix = decoderNetwork.decoder_embedding.variables[0] 
(first_finished, first_inputs,first_state) = decoder_instance.initialize(decoder_embedding_matrix,
                             start_tokens = start_tokens,
                             end_token=end_token,
                             initial_state = decoder_initial_state)
#print( first_finished.shape)
print(f"first_inputs returns the same decoder_input i.e. embedding of  {SOS} : {first_inputs.shape}")
print(f"start_index_emb_avg {tf.reduce_sum(tf.reduce_mean(first_inputs, axis=0))}") # mean along the batch

inputs = first_inputs
state = first_state  
predictions = np.empty((inference_batch_size,0), dtype = np.int32)                                                                             
for j in range(maximum_iterations):
    outputs, next_state, next_inputs, finished = decoder_instance.step(j,inputs,state)
    inputs = next_inputs
    state = next_state
    outputs = np.expand_dims(outputs.sample_id,axis = -1)
    predictions = np.append(predictions, outputs, axis = -1)

a_tx : (1, 1024)
c_tx : (1, 1024)
decoder_initial_state = [a_tx, c_tx] : (2, 1, 1024)

Compared to simple encoder-decoder without attention, the decoder_initial_state
is an AttentionWrapperState object containing s_prev tensors and context and alignment vector

decoder initial state shape: (6,)
decoder_initial_state tensor
AttentionWrapperState(cell_state=[<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[-0.02423859, -0.04404252, -0.08185287, ..., -0.00134279,
         0.07152605,  0.01424581]], dtype=float32)>, <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[-0.05931591, -0.10267326, -0.21703617, ..., -0.00413214,
         0.1776058 ,  0.0562396 ]], dtype=float32)>], attention=<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, time=<tf.Tensor: shape=(), dtype=int32, numpy=0>, alignments=<tf.Tensor: shape=(1, 8), dtype=float32, numpy=array([[0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>, alignment_history=(

In [ ]:
#prediction based on our sentence earlier
print("English Sentence:")
print(input_raw)
print("\nGerman Translation:")
for i in range(len(predictions)):
    line = predictions[i,:]
    seq = list(itertools.takewhile( lambda index: index !=2, line))
    print(" ".join( [ge_tokenizer.index_word[w] for w in seq]))

English Sentence:
how are you

German Translation:
sie sind wenig
